# Suitability Analysis: Best place within the Boston Region for a Tufts UEP student and BU Law student to live without a car 
UEP-239 Final Project\
By: Justina Cheng

[DESCRIPTION]

## Import Dependencies

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import osmnx as ox
import networkx as nx
from geopy.geocoders import Nominatim
from pyproj import CRS
from shapely.geometry import LineString, Point, Polygon, box

import rasterio
from rasterio.plot import show
from rasterio import features

import richdem as rd
from scipy import ndimage
from rasterstats import zonal_stats

## Create and View Base Map of Boston Region Zip Code Tabulation Areas (ZCTAs)
To create a GeoDataFrame of the Boston Region ZCTAs, the following steps were used:
1. Massachusetts outline with detailed coastline was imported from MassGIS as a GeoDataFrame.
1. Massachusetts ZCTAs were imported from the Census Bureau as a GeoDataFrame.
1. The outline and ZCTAs GeoDataFrames were converted to the coordinate reference system (CRS) for the Massachusetts Mainland EPSG 6491.
1. Boundaries for the Boston Region Metropolitan Planning Organization (MPO) were imported from MassDOT as a GeoDataFrame, and the CRS was converted to EPSG 6491.
1. The Boston Region was extracted from the MPO.
1. Massachusetts ZCTAs within the Boston Region were extracted using the centroid of the ZCTAs.
1. Function `convert_n_clip` was created to convert a GDF to the CRS of another GDF and clip to the other's extent.
1. Function `read_n_clip` was created to read in a shapefile and use `convert_n_clip` to convert it to the CRS of another GDF and clip to the other's extent.
1. Massachusetts Surface Water data from MassGIS was processed with `read_n_clip` with the extent of Boston ZCTAs. 

### Massachusetts Coastline

In [ ]:
# Import outline of detailed Massachusetts coastline.
outline_25k = gpd.read_file("./data/outline25k/OUTLINE25K_POLY.shp")
outline_25k.info()

In [ ]:
# View CRS and plot.
print(outline_25k.crs)
outline_25k.plot(figsize=(12,12))
plt.title('Massachusetts Detailed Coastline', fontsize=16)
plt.show()

### Massachusetts ZCTAs

In [ ]:
# Import Zip Code Tabulation Areas within Massachusetts.
ma_zcta = gpd.read_file("./data/tl_2010_25_zcta500/tl_2010_25_zcta500.shp")
ma_zcta.info()

In [ ]:
# View CRS and plot.
print(ma_zcta.crs)
ma_zcta.plot(figsize=(12,12))
plt.title('Massachusetts ZCTAs', fontsize=16)
plt.show()

In [ ]:
# Convert CRS to Massachusetts Mainland EPSG 6491.
outline_25k = outline_25k.to_crs('epsg:6491')
ma_zcta = ma_zcta.to_crs('epsg:6491')
# Confirm CRSs match.
outline_25k.crs == ma_zcta.crs

In [ ]:
# Clip ZCTA GDF to 25k MA outline.
ma_zcta_25k = gpd.clip(ma_zcta, outline_25k)
ma_zcta_25k.info()

### Boston Region Metropolitan Planning Organization (MPO)

In [ ]:
# Import boundaries from Boston Region Metropolitan Planning Organization.
mpo = gpd.read_file("./data/MPO_Boundaries/MPO_Boundaries.shp")
mpo.info()

In [ ]:
# View MPO dataset.
mpo

In [ ]:
# Convert MPO CRS to EPSG 6491 and plot.
mpo = mpo.to_crs('epsg:6491')
mpo.plot(figsize=(12,12))
plt.title('Boston Region MPO Boundaries', fontsize=16)
plt.show()

In [ ]:
# Extract only Boston Region from MPO.
boston_region = mpo.loc[mpo.MPO == 'Boston Region'].reset_index()
boston_region

In [ ]:
# Extract ZCTAs within the Boston Region using the centroid of the ZCTAs.
boston_zcta = ma_zcta_25k[ma_zcta_25k.centroid.within(boston_region.geometry[0])].reset_index()
boston_zcta.info()

In [ ]:
# View the Boston Region ZCTAs.
boston_zcta

In [ ]:
# Plot the Boston Region ZCTAs.
boston_zcta.plot(figsize=(12,12))
plt.title('ZCTAs within Boston Region', fontsize=16)
plt.show()

### Define Functions `convert_n_clip` and `read_n_clip`
`convert_n_clip` takes two GeoDataFrames (GDF): one to process (gdf) and one whose extent will be used to clip. The function converts the coordinate reference system (CRS) of the original GDF and clips it to the extent of the extent GDF.

In [ ]:
def convert_n_clip(orig_gdf, extent_gdf):
    """
    Takes two GeoDataFrames (GDF): one to process (orig_gdf) and one whose extent will be used to clip (extent_gdf).
    Converts the coordinate reference system (CRS) of the orig_gdf to the CRS of extent_gdf.
    Clips to the extent of orig_gdf to the extent of extent_gdf.
    Returns clipped GDF.
    
    Inputs:
    orig_gdf = GDF to process
    extent_gdf = GDF whose extent to use
    
    Example:
    ma_schools = convert_n_clip(usa_schools, ma_boundary)
    """
    orig_gdf = orig_gdf.to_crs(extent_gdf.crs)
    clipped_gdf = gpd.clip(orig_gdf, extent_gdf)
    return clipped_gdf   

`read_n_clip` takes a filepath for a shapefile and a GDF whose extent will be used to clip the shapefile. The function reads in the shapefile and uses `convert_n_clip` to convert the coordinate reference system (CRS) of the original GDF and clip it to the extent of the extent GDF.

In [ ]:
def read_n_clip(filepath, extent_gdf):
    """
    Takes a filepath for a shapefile and a GeoDataFrame (GDF).
    Reads in the file.
    Uses convert_n_clip function to convert to the coordinate reference system (CRS)
    of the GDF and clip to the extent of the GDF. 
    Returns clipped GDF.
    
    Inputs:
    filepath = relative filepath for shapefile to read
    extent_gdf = GDF whose extent to use
    
    Example:
    ma_water = read_n_clip('./data/usa/water.shp', ma_boundary)
    """
    shapefile = gpd.read_file(filepath)
    clipped_shapefile = convert_n_clip(shapefile, extent_gdf)
    return clipped_shapefile

In [ ]:
# read_n_clip Boston surface water.
boston_water = read_n_clip('./data/hydro25k/HYDRO25K_POLY.shp', boston_zcta)
print(boston_water.crs)
boston_water.info()

In [ ]:
# Plot the Boston Region ZCTAs with surface water.
ax = boston_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax)
plt.title('ZCTAs and Waterways within Boston Region', fontsize=16)
plt.show()

## Find Tufts University and Boston University Locations
To find the locations of Tufts University and Boston University (BU), the Massachusetts Colleges and Universities shapefile was processed with `read_n_clip` to read the shapefile and clip it to the extent of `boston_zcta`. Tufts and BU were then extracted into a GeoDataGrame.

In [ ]:
# read_n_clip Boston MPO colleges.
colleges = read_n_clip('./data/colleges/COLLEGES_PT.shp', boston_zcta)
print(colleges.crs)
colleges.info()

In [ ]:
# Plot Boston MPO colleges.
ax = boston_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax, alpha=0.3)
colleges.plot(ax=ax, color='maroon')
plt.title('Colleges in Boston Region MPO', fontsize=16)
plt.show()

In [ ]:
# List all college names.
college_list = list(colleges.COLLEGE.unique())
college_list

In [ ]:
# Select only names matching Tufts University or Boston University.
colleges_select = colleges.loc[colleges.COLLEGE.isin(['Tufts University', 'Boston University'])]
colleges_select

In [ ]:
# Select only the Medford/Somerville Tufts Campus and the main BU Campus.
tufts_bu = colleges_select.iloc[[0, 1]]
tufts_bu

In [ ]:
# Plot Tufts and BU.
ax = boston_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax, alpha=0.3)
tufts_bu.plot(ax=ax, color='maroon', markersize=50)
plt.title('Tufts University and Boston Universtiy Over Boston Region MPO', fontsize=16)
plt.show()

## Import Mass Transit Stops and Routes

### MBTA Bus Stops

In [ ]:
# read_n_clip MBTA bus stops and view info.
bos_bus = read_n_clip('./data/MBTA_Bus_Routes_and_Stops/MBTA_Bus_Routes_and_Stops.shp', boston_zcta)
print(bos_bus.crs)
bos_bus.info()

### MBTA Rapid Transit (T) Stops and Routes

In [ ]:
# read_n_clip MBTA rapid transit (T) stops and view info.
bos_rt_node = read_n_clip('./data/mbta_rapid_transit/MBTA_NODE.shp', boston_zcta)
print(bos_rt_node.crs)
bos_rt_node.info()

In [ ]:
# read_n_clip MBTA rapid transit (T) routes and view info.
bos_rt_route = read_n_clip('./data/mbta_rapid_transit/MBTA_ARC.shp', boston_zcta)
print(bos_rt_route.crs)
bos_rt_route.info()

### Commuter Rail Stops and Routes

In [ ]:
# read_n_clip Commuter Rail stops and view info.
bos_train_node = read_n_clip('./data/trains/TRAINS_NODE.shp', boston_zcta)
print(bos_train_node.crs)
bos_train_node.info()

In [ ]:
# read_n_clip Commuter Rail routes and view info.
bos_train_route = read_n_clip('./data/trains/TRAINS_RTE_TRAIN.shp', boston_zcta)
print(bos_train_route.crs)
bos_train_route.info()

### Map Mass Transit with Base Map

In [ ]:
ax = boston_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax, alpha=0.3)
bos_bus.plot(ax=ax, color='red', markersize=5, label='Bus Stop')
bos_rt_node.plot(ax=ax, color='green', markersize=15, label='Rapid Transit Stop')
bos_rt_route.plot(ax=ax, color='green', label='Rapid Transit Route')
bos_train_node.plot(ax=ax, color='purple', markersize=20, label='Train Stop')
bos_train_route.plot(ax=ax, color='purple', label='Train Route')
tufts_bu.plot(ax=ax, color='maroon', markersize=50)
plt.title('Mass Transit in Boston Region MPO', fontsize=16)
plt.legend()
plt.show()

## Limit Study Area to Extent of Rapid Transit
Judging by the vast extent of mass transit and the locations of Tufts and BU, the outer ZCTAs within the Boston Region MPO are untenable for regular commutes to campus. The T is more commonly (and reasonably) used for commuting. To limit the study area to the extent of the T for a more realistic comparison of ZCTAs, the following steps were used:
1. Extract the rectangular bounds of MBTA Rapid Transit (T) stops.
1. Create a bounding box with `shapely.geometry.box`.
1. Add a buffer to the bounding box and store as a new extent.
1. Extract Boston Region ZCTAs whose centroids are within the extent.

In [ ]:
# Extract bounds of Boston Rapid Transit (T) nodes.
rt_bounds = bos_rt_node.geometry.total_bounds
rt_bounds

In [ ]:
# Creating bounding box with shapely.geometry.box
# shapely.geometry.box(minx, miny, maxx, maxy, ccw=True)
rt_bound_box = box(rt_bounds[0], rt_bounds[1], rt_bounds[2], rt_bounds[3])
rt_bound_box

In [ ]:
# Store the extent as a Shapely Polygon in a variable called graph_extent.
graph_extent = rt_bound_box.buffer(0.1, join_style=2)
graph_extent

In [ ]:
# Extract Boston Region ZCTAs within the graph extent using the centroid of the ZCTAs.
rt_zcta = boston_zcta[boston_zcta.centroid.within(graph_extent)]
rt_zcta.info()

In [ ]:
# View first five rows of rt_zcta.
rt_zcta.head()

In [ ]:
# Plot the Boston Region ZCTAs within graph extent to confirm success.
ax = rt_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
bos_rt_node.plot(ax=ax, color='green', markersize=15, label='Rapid Transit Stop')
bos_rt_route.plot(ax=ax, color='green', label='Rapid Transit Route')
plt.title('ZCTAs within Boston Region Rapid Transit Extent', fontsize=16)
plt.show()

### Clip All Relevant GDFs
The following GDFs were clipped to the new `rt_zcta` extent:
- `boston_water`
- `bos_bus`
- `bos_train_node`
- `bos_train_route`

Because `bos_rt_node` was used to create the extent and `bos_rt_route` connects all T stops, `bos_rt_route` does not need to be clipped.

In [ ]:
# Clip all relevant GDFs.
boston_water = gpd.clip(boston_water, rt_zcta)
bos_bus = gpd.clip(bos_bus, rt_zcta)
bos_train_route = gpd.clip(bos_train_route, rt_zcta)
bos_train_node = gpd.clip(bos_train_node, rt_zcta)

In [ ]:
# Plot new extent with mass transit and schools.
ax = rt_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax, alpha=0.3)
bos_bus.plot(ax=ax, color='red', markersize=1, label='Bus Stop')
bos_rt_node.plot(ax=ax, color='green', markersize=15, label='Rapid Transit Stop')
bos_rt_route.plot(ax=ax, color='green', label='Rapid Transit Route')
bos_train_node.plot(ax=ax, color='purple', markersize=20, label='Train Stop')
bos_train_route.plot(ax=ax, color='purple', label='Train Route')
tufts_bu.plot(ax=ax, color='maroon', markersize=50)
plt.title('Mass Transit within Boston Region Rapid Transit Extent', fontsize=16)
plt.legend()
plt.show()

#### TEST CODE

bos_bus.head()

bos_bus_latlong = bos_bus.to_crs('epsg:4326')
bos_bus_latlong

bos_bus['latitude'] = bos_bus_latlong.geometry.y
bos_bus['longitude'] = bos_bus_latlong.geometry.x
bos_bus.head()

m_1 = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=20)
mc_1 = MarkerCluster()
for idx, row in bos_bus.iterrows():
    mc_1.add_child(folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(color='red', icon='bus')))

mc_1.add_to(m_1)
m_1

## Analyze Mass Transit Stops
As Tufts University and Boston University are fairly far walking distance from one another, it is crucial that the home's location be easily accessible by mass transit. Transit stops were selected as an indicator for convenience of mass transit. While the routes are also important to consider, the stops are the on-off points to transit lines and necessary to accessing the transit systems.

The density of mass transit stops was calculated with the following steps:
1. Function `count_records` was created to count the number of records in a GDF within polygons of another GDF.
1. `count_records` was used on the GDFs for bus stops, T stops, and train stops in the limited `rt_zcta` extent.
1. Function `multimerge` was created to merge multiple DataFrames on the same column or list of columns. 
1. `multimerge` was used to add all mass transit stop counts to `rt_zcta` 
1. Total stops and stop density were calculated for all ZCTAs and mapped.

### Count Public Transit Nodes per ZCTA

#### Define Function `count_records`
`count_records` takes a GeoDataFrame of records to count and a GDF of polygons to count how many records are within each polygon. It outputs a DataFrame with the specified polygon column and counts. The optional argument `op` for `gpd.sjoin` defaults to `'within'` unless otherwise specified.

In [ ]:
def count_records(records_gdf, polygon_gdf, polygon_col, count_col, op='within'):
    """
    Takes a GeoDataFrame of records to count and a GDF of polygons to count how 
    many records are within each polygon. 
    Outputs a DataFrame with the specified polygon column and counts.
    Optional argument op defaults to 'within' unless otherwise specified.
    
    Inputs:
    records_gdf = GDF of records to count
    polygon_gdf = GDF of polygons to count from
    polygon_col = name of column in polygon_gdf
        e.g. 'name'
    count_col = name of column for counts in output
        e.g. 'tree_count'
    op = op argument for sjoin; defaults to 'within' unless otherwise specified
    
    Example:
    tree_count = count_records(trees, towns, 'name', 'tree_count')
    """
    spatial_join = gpd.sjoin(records_gdf, polygon_gdf, how='left', op=op)
    records_count = spatial_join[polygon_col].value_counts().reset_index()
    records_count.columns = [polygon_col, count_col]
    return records_count

#### Count MBTA Bus Stops per ZCTA

In [ ]:
# count_records for bos_bus within rt_zcta.
zcta_bus_count = count_records(bos_bus, rt_zcta, 'ZCTA5CE00', 'bus_stop_count')
zcta_bus_count.describe()

#### Count MBTA T Stops per ZCTA

In [ ]:
# count_records for bos_rt_node within rt_zcta.
zcta_rt_count = count_records(bos_rt_node, rt_zcta, 'ZCTA5CE00', 'rt_stop_count')
zcta_rt_count.describe()

#### Count Commuter Rail Stops per ZCTA

In [ ]:
# count_records for bos_train_node within rt_zcta.
zcta_train_count = count_records(bos_train_node, rt_zcta, 'ZCTA5CE00', 'train_stop_count')
zcta_train_count.describe()

### Define Function `multimerge`
`multimerge` takes a base DataFrame (DF) and merges it with each DF in a list of DataFrames on the specified column or list of columns and with the specified 'how'. The function assumes the specified column(s) exist(s) across all DFs.

In [ ]:
def multimerge(left_df, df_list, on_col, how):
    """
    Takes a base DataFrame and merges with a list of DataFrames on the 
    specified column or list of columns and with the specified 'how'.
    Assumes on_col exists across all DFs.
    
    Inputs:
    left_df = base DF
    df_list = list of DFS
        e.g. [df1, df2, df3]
    on_col = bracketed column name or list of columns (same across DFs)
        e.g. ['name'], ['name', 'address']
    how = how argument
        e.g. 'left'
    
    Example:
    town_schools = multimerge(town, [elem, middle, high], ['town_name'], 'left')
    """
    merge_df = left_df.copy()
    for i in range(len(df_list)):
        merge_df = merge_df.merge(df_list[i], on=on_col, how=how)
    return merge_df

### Calculate Mass Transit Density per ZCTA

In [ ]:
# Merge rt_zcta with all transit stop counts
count_list = [zcta_bus_count, zcta_rt_count, zcta_train_count]
zcta_nodes = multimerge(rt_zcta, count_list, ['ZCTA5CE00'], 'left').fillna(0)
zcta_nodes.info()

In [ ]:
# View first five rows of new GDF.
zcta_nodes.head()

In [ ]:
# Calculate total transit stops in each ZCTA.
zcta_nodes['nodes_count'] = zcta_nodes.bus_stop_count + zcta_nodes.rt_stop_count + zcta_nodes.train_stop_count
zcta_nodes

In [ ]:
# Map the number of transit stops in each ZCTA.
ax = zcta_nodes.plot(column = 'nodes_count',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Number of Mass Transit Stops"})
plt.title('Number of Mass Transit Stops by ZCTA in Boston Region', fontsize = 20)
plt.show()

In [ ]:
# Calculate node density in nodes/sqkm.
zcta_nodes['nodes_density'] = zcta_nodes.nodes_count/zcta_nodes.area*(10**6)
zcta_nodes.sort_values(by='nodes_density', ascending=False).head()

In [ ]:
# View statistics for nodes_density.
zcta_nodes.nodes_density.describe()

The top value for `nodes_density` greatly exceeds the next value, despite having a low `nodes_count`, indicating it is an outlier. The ZCTA in question, ZCTA 02222, appears to contain only TD Garden and North Station. The decision was made to set the `nodes_density` value for ZCTA 02222 to the median value to prevent skewing the analysis.

In [ ]:
# ZCTA 02222 value for nodes_density set to median value.
zcta_nodes.loc[68, 'nodes_density'] = zcta_nodes.nodes_density.median()
zcta_nodes.sort_values(by='nodes_density', ascending=False).head()

In [ ]:
# View statistics for nodes_density.
zcta_nodes.nodes_density.describe()

In [ ]:
# Map the density of transit nodes in each ZCTA.
ax = zcta_nodes.plot(column = 'nodes_density',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Mass Transit Stops per sqkm"})
plt.title('Density of Mass Transit Stops by ZCTA in Boston Region', fontsize = 20)
plt.show()

## Locations and Density of Necessities
Accessibility of necessities and amenities is crucial to living anywhere. For the purposes of this study, necessities were defined as follows:
- Food
    - Groceries (e.g. supermarkets and food purveryors, like  greengrocers, butchers, etc.)
    - Prepared food (e.g. restaurants, cafes, etc.)
- Health Services
    - Community health centers
    - Hospitals
    - Healthcare facilities (e.g. doctors' offices, pharmacies, dentists, etc.)
- Public Services
    - Fire stations
    - Police stations
    - USPS Post Offices

Some data was imported from sources such as MassGIS while others were retrieved using `OpenStreetMap` and `OSMnx`'s `geometries_from_polygon` function.

### Create Extent in Latitude-Longitude to Use With `OSMnx`
To use `OSMnx geometries_from_polygon`, a polygon needs to be created in latitude-longitude coordinates. This was accomplished with the following steps:
1. Convert the T stops shapefile to `EPSG:4326` for lat-long and extract its rectangular bounds.
1. Create a bounding box with `shapely.geometry.box`.
1. Add a buffer to the bounding box and store as a new extent.

In [ ]:
# Extract bounds of T Stops.
rt_bounds_latlong = bos_rt_node.to_crs('epsg:4326').geometry.total_bounds
rt_bounds_latlong

In [ ]:
# Creating bounding box with shapely.geometry.box
# shapely.geometry.box(minx, miny, maxx, maxy, ccw=True)
rt_bound_box_latlong = box(rt_bounds_latlong[0], rt_bounds_latlong[1], rt_bounds_latlong[2], rt_bounds_latlong[3])
rt_bound_box_latlong

In [ ]:
# Store the extent as a Shapely Polygon in a variable called graph_extent:
graph_extent_latlong = rt_bound_box_latlong.buffer(0.1, join_style=2)
print(type(graph_extent_latlong))
graph_extent_latlong

### Food Within Extent

#### Groceries

In [ ]:
# Retrieve groceries features within graph_extent_latlong from OSMnx and view info.
grocery_tags = {'shop':['supermarket', 'greengrocer', 'bakery', 'butcher', 'deli', 'dairy', 'farm', 'seafood']}
grocery = ox.geometries_from_polygon(graph_extent_latlong, grocery_tags)
grocery = convert_n_clip(grocery, rt_zcta)
grocery.info()

In [ ]:
# View first five rows of groceries GDF.
grocery.head()

In [ ]:
# Use count_records on groceries GDF.
zcta_grocery_count = count_records(grocery, rt_zcta, 'ZCTA5CE00', 'grocery_count')
zcta_grocery_count

#### Prepared food

In [ ]:
# Retrieve prepared food features within graph_extent_latlong from OSMnx and view info.
prep_food_tags = {'amenity':['cafe', 'restaurant']}
prep_food = ox.geometries_from_polygon(graph_extent_latlong, prep_food_tags)
prep_food = convert_n_clip(prep_food, rt_zcta)
prep_food.info()

In [ ]:
# View first five rows of prepared food GDF.
prep_food.head()

In [ ]:
# Use count_records on prepared food GDF.
zcta_prep_food_count = count_records(prep_food, rt_zcta, 'ZCTA5CE00', 'prep_food_count')
zcta_prep_food_count

#### Calculate Food Density per ZCTA

In [ ]:
# Merge rt_zcta with all food counts.
count_list = [zcta_grocery_count, zcta_prep_food_count]
zcta_food = multimerge(rt_zcta, count_list, ['ZCTA5CE00'], 'left').fillna(0)
zcta_food.head()

In [ ]:
# Calculate total food establishment in each ZCTA.
zcta_food['food_count'] = zcta_food.grocery_count + zcta_food.prep_food_count
# Calculate food density in establishments/sqkm.
zcta_food['food_density'] = zcta_food.food_count/zcta_food.area*(10**6)
zcta_food.sort_values(by='food_density', ascending=False).head()

##### TEST CODE `calc_density`

In [1]:
def calc_density(orig_gdf, gdf_count_list, on_col, how, count_cols, total_count_col, density_col):
    """
    The purpose of this function is to calculate the density of a set of records
    in a GeoDataFrame of polygons.
    Takes a base GeoDataFrame and, using function multimerge, merges it with a list of GeoDataFrames 
    on the specified column or list of columns and with the specified 'how'.
    Adds together the counts in specified columns, then calculates the density.
    Assumes on_col exists across all DFs.
    Assumes units of the GDF are in meters, with density output of per sqkm.
    
    Inputs:
    orig_gdf = base GDF
    gdf_count_list = list of GDFs with counts to calculate from
        e.g. [gdf1, gdf2, gdf3]
    on_col = bracketed column name or list of columns (same across GDFs)
        e.g. ['name'], ['name', 'address']
    how = how argument
        e.g. 'left'
    count_cols = bracketed list of column names with counts corresponding to gdf_count_list
        e.g. ['gdf1_count', 'gdf2_count', 'gdf3_count']
    total_count_col = string name for new column with total counts
        e.g. 'total_count'
    density_col = string name for new column with density (per sqkm)
        e.g. 'bike_density'
    
    Example:
    schools = [elem, middle, high]
    on_col = ['town_name', 'state']
    count_cols = ['elem_classrooms', 'middle_classrooms', 'high_classrooms']
    town_schools = multimerge(town, schools, on_col, 'left', count_cols, 'total_classrooms', 'classroom_density')
    """
    density_gdf = multimerge(orig_gdf, gdf_count_list, on_col, how).fillna(0)
    density_gdf[total_count_col] = ""
    for i in range(len(count_cols)):
        density_gdf[total_count_col] = density_gdf[total_count_col] + density_gdf[count_cols[i]]
    density_gdf[density_col] = density_gdf[total_count_col]/density_gdf.area*(10**6)
    return density_gdf

In [ ]:
# Map the density of food establishments in each ZCTA.
ax = zcta_food.plot(column = 'food_density',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Food establishments per sqkm"})
plt.title('Density of Food Establishments by ZCTA in Boston Region', fontsize = 20)
plt.show()

### Health services within extent

#### Community Health Centers

In [ ]:
# read_n_clip Community Health Centers shapefile from MassGIS and view info.
comm_health = read_n_clip('./data/chcs/CHCS_PT.shp', rt_zcta)
comm_health.info()

In [ ]:
# View first five rows of comm_health.
comm_health.head()

In [ ]:
# Use count_records on comm_health.
zcta_comm_health_count = count_records(comm_health, rt_zcta, 'ZCTA5CE00', 'comm_health_count')
zcta_comm_health_count.info()

#### Hospitals

In [ ]:
# read_n_clip Hospitals shapefile from MassGIS and view info.
hospitals = read_n_clip('./data/acute_care_hospitals/HOSPITALS_PT.shp', rt_zcta)
hospitals.info()

In [ ]:
# View first five rows of hospitals.
hospitals.head()

In [ ]:
# Use count_records on hospitals.
zcta_hospitals_count = count_records(hospitals, rt_zcta, 'ZCTA5CE00', 'hospitals_count')
zcta_hospitals_count.info()

#### Healthcare

In [ ]:
# Retrieve healthcare features within graph_extent_latlong from OSMnx and view info.
health_tags = {'healthcare':True, 'amenity':['clinic', 'doctors', 'dentist', 'health_post', 'pharmacy']}
healthcare = ox.geometries_from_polygon(graph_extent_latlong, health_tags)
healthcare = convert_n_clip(healthcare, rt_zcta)
healthcare.info()

In [ ]:
# View first five rows of healthcare GDF.
healthcare.head()

In [ ]:
# Use count_records on healthcare.
zcta_healthcare_count = count_records(healthcare, rt_zcta, 'ZCTA5CE00', 'healthcare_count')
zcta_healthcare_count.info()

#### Calculate health services density per ZCTA

In [ ]:
count_list = [zcta_comm_health_count, zcta_hospitals_count, zcta_healthcare_count]
zcta_health = multimerge(rt_zcta, count_list, ['ZCTA5CE00'], 'left').fillna(0)
zcta_health.head()

In [ ]:
zcta_health['health_count'] = zcta_health.comm_health_count + zcta_health.hospitals_count + zcta_health.healthcare_count
zcta_health['health_density'] = zcta_health.health_count/zcta_health.area*(10**6)
zcta_health.sort_values(by='health_density', ascending=False).head()

In [ ]:
# Map the density of transit nodes in each ZCTA.
ax = zcta_health.plot(column = 'health_density',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Health services per sqkm"})
plt.title('Density of Health Services by ZCTA in Boston Region', fontsize = 20)
plt.show()

### Public services within extent

#### Fire Stations

In [ ]:
# read_n_clip firestations shapefile from MassGIS and view info.
fire = read_n_clip('./data/firestations_pt/FIRESTATIONS_PT_MEMA.shp', rt_zcta)
fire.info()

In [ ]:
# View first five rows of fire GDF.
fire.head()

In [ ]:
# Use count_records on fire.
zcta_fire_count = count_records(fire, rt_zcta, 'ZCTA5CE00', 'fire_count')
zcta_fire_count.info()

#### Police Stations

In [ ]:
# read_n_clip police stations shapefile from MassGIS and view info.
police = read_n_clip('./data/policestations/POLICESTATIONS_PT_MEMA.shp', rt_zcta)
police.info()

In [ ]:
# View first five rows of police GDF.
police.head()

In [ ]:
# Use count_records on police.
zcta_police_count = count_records(police, rt_zcta, 'ZCTA5CE00', 'police_count')
zcta_police_count.info()

#### USPS Post Offices

In [ ]:
# Read in CSV files of USPS DDUs (Destination Delivery Units) obtained from USPS
usps_df = pd.read_csv('./data/USPS DDUs.csv')
# Convert to GeoDataFrame, setting CRS to WGS 84 Pseudo Mercator EPSG:3857
usps = gpd.GeoDataFrame(usps_df, geometry=gpd.points_from_xy(usps_df.x, usps_df.y))
usps = usps.set_crs('epsg:3857')

In [ ]:
# View usps GDF.
usps

In [ ]:
# convert_n_clip usps using rt_zcta
usps = convert_n_clip(usps, rt_zcta)
usps.info()

In [ ]:
# View first five rows of usps GDF.
usps.head()

In [ ]:
# Use count_records on usps.
zcta_usps_count = count_records(usps, rt_zcta, 'ZCTA5CE00', 'usps_count')
zcta_usps_count.info()

#### Calculate Public Services density per ZCTA

In [ ]:
count_list = [zcta_fire_count, zcta_police_count, zcta_usps_count]
zcta_public_service = multimerge(rt_zcta, count_list, ['ZCTA5CE00'], 'left').fillna(0)
zcta_public_service.head()

In [ ]:
zcta_public_service['public_service_count'] = zcta_public_service.fire_count + zcta_public_service.police_count + zcta_public_service.usps_count
zcta_public_service['public_service_density'] = zcta_public_service.public_service_count/zcta_public_service.area*(10**6)
zcta_public_service.sort_values(by='public_service_density', ascending=False).head()

In [ ]:
# Map the density of transit nodes in each ZCTA.
ax = zcta_public_service.plot(column = 'public_service_density',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Public services per sqkm"})
plt.title('Density of Public Services by ZCTA in Boston Region', fontsize = 20)
plt.show()

In [ ]:
ax = rt_zcta.plot(color='lightgrey', edgecolor='grey', figsize=(12,12))
boston_water.plot(ax=ax, alpha=0.3)
grocery.plot(ax=ax, markersize=15, color='yellow', label='Groceries')
healthcare.plot(ax=ax, markersize=15, color='aquamarine', label='Healthcare')
comm_health.plot(ax=ax, markersize=15, color='coral', label='Community Health Centers')
hospitals.plot(ax=ax, markersize=20, color='orangered', label='Hospitals')
public_service.plot(ax=ax, markersize=10, color='dodgerblue', label='Public Services')
plt.title('Necessary Amenities within Boston Region Rapid Transit Extent', fontsize=16)
plt.legend()
plt.show()

### Necessities/Amenities per ZCTA

In [ ]:
necs_list = [zcta_food, zcta_health, zcta_public_service]
cols_list = ['index', 'STATEFP00', 'ZCTA5CE00', 'GEOID00', 'CLASSFP00', 'MTFCC00', 'FUNCSTAT00', 'ALAND00', 'AWATER00', 'INTPTLAT00', 'INTPTLON00', 'PARTFLG00', 'geometry']
zcta_necs = multimerge(rt_zcta, necs_list, on_col=cols_list, how='left')
zcta_necs

In [ ]:
zcta_necs.columns

In [ ]:
zcta_necs['necs_count'] = zcta_necs.food_count + zcta_necs.health_count + zcta_necs.public_service_count
zcta_necs['necs_density'] = zcta_necs.necs_count/zcta_necs.area*(10**6)
zcta_necs.sort_values(by='necs_density', ascending=False).head()

In [ ]:
# Map the density of necessities in each ZCTA.
ax = zcta_necs.plot(column = 'necs_density',
                      legend = True,
                      edgecolor = 'black',
                      cmap = 'OrRd',
                      figsize = (12, 12),
                      legend_kwds={'label': "Necessities per sqkm"})
plt.title('Density of Necessities by ZCTA in Boston Region', fontsize = 20)
plt.show()

## Rent Affordability

## Bike Facilities

## Leisure